In [79]:
import numpy as np
import re

## Adapted from E. Prud'hommeaux with updates.
## You are provided the code you need except the Viterbi algorithm implementation (find optimal path). 
## Note that the files with probabilites are *not* log probabilities.
## Add your code where it says "YOUR CODE GOES HERE".


######################################
### READ IN EMISSION PROBABILITIES ###
######################################

## For a given POS, how likely is each word? Stored in a dict where the keys are POS and the value is 
## a dictionary of word:prob pairs. Inspect to ensure the probabilities make sense.

emits = {}  ## stores emission probabilities
posfrequency=defaultdict(int) # relative POS frequency for OOV to pos transition
f = open('emissProbs.txt')
for line in f:
    parts = line.rstrip().split()
    pos = parts[0]
    word = parts[1]
    eprob = float(parts[2])
    if pos in emits.keys():
        posfrequency[pos]+=1
        emits[pos][word] = eprob
    else:
        emits[pos] = {word:eprob}
f.close()
print(emits)

{'BOS': {'BOS': 1.0}, 'NN': {'gardening': 3.31707964308223e-05, 'scramble': 6.63415928616446e-05, 'Speculation': 6.63415928616446e-05, 'B.A.T': 6.63415928616446e-05, 'danger': 0.0001990247785849338, 'channel': 3.31707964308223e-05, 'reluctance': 3.31707964308223e-05, 'chlorazepate': 3.31707964308223e-05, 'reputation': 0.0001326831857232892, 'billionaire': 6.63415928616446e-05, 'blanket': 6.63415928616446e-05, 'boat': 3.31707964308223e-05, 'miscommunication': 3.31707964308223e-05, '55th': 3.31707964308223e-05, 'hobby': 3.31707964308223e-05, 'reflection': 3.31707964308223e-05, 'stockbroker': 3.31707964308223e-05, 'street': 0.0002653663714465784, 'sovereignty': 9.95123892924669e-05, 'viability': 3.31707964308223e-05, 'advantage': 0.0005307327428931568, 'naturalist': 3.31707964308223e-05, 'news': 0.0020234185822801606, 'leadership': 0.0005639035393239792, 'insurer': 0.0003980495571698676, 'implication': 6.63415928616446e-05, 'landmark': 3.31707964308223e-05, 'product': 0.001393173450094536

In [80]:
########################################
### READ IN TRANSITION PROBABILITIES ###
########################################

## How likely is one POS to follow another POS? Stored in a dict where keys are tuples of POS sequences (a tag followed by next tag)
## and the values are the probabilities of those sequences. Inspect to ensure the probabilities make sense.

transitions = {}  ## stores transition probabilities

f = open('transitProbs.txt')
for line in f:
    parts = line.rstrip().split()
    pos1 = parts[0]
    pos2 = parts[1]
    prob = float(parts[2])
    transitions[(pos1, pos2)] = prob
f.close()

# relative frequency of pos tags give us relative probability of transition from an OOV word to each POS,
# for pos to OOV, use default.
count=0
for f in posfrequency:
    count+=posfrequency[f]

defaultprob =  0.00000000001
for p in emits.keys():
    transitions[('OOV', p)] = posfrequency[p]/count
    transitions[(p, 'OOV')] = defaultprob
transitions[('OOV', 'OOV')] = defaultprob

# for p in emits.keys():
#     print(transitions[p,'OOV'])


print(posfrequency)
print(transitions)

defaultdict(<class 'int'>, {'NN': 4023, 'IN': 133, 'DT': 34, 'VBZ': 505, 'RB': 609, 'VBN': 1054, 'TO': 3, 'VB': 1100, 'JJ': 2718, 'NNS': 2230, 'NNP': 4151, 'CC': 14, 'POS': 1, '.': 2, 'VBP': 420, 'VBG': 899, 'PRP$': 13, 'CD': 1828, '``': 1, "''": 1, 'VBD': 766, 'EX': 1, 'MD': 18, '(': 1, '$': 4, ')': 1, 'NNPS': 126, 'PRP': 33, 'JJS': 53, 'WP': 6, 'RBR': 18, 'JJR': 75, 'WDT': 5, 'WRB': 10, 'RBS': 4, 'PDT': 1, 'RP': 1, ':': 4, 'FW': 20, 'SYM': 2, 'UH': 7, 'BOS': 0, ',': 0, 'EOS': 0, '#': 0, 'WP$': 0})
{('BOS', 'NN'): 0.045210384959713516, ('NN', 'IN'): 0.24771950774538096, ('IN', 'DT'): 0.3261729045861887, ('DT', 'NN'): 0.48453776929370057, ('NN', 'VBZ'): 0.038776661027631276, ('VBZ', 'RB'): 0.13446643717728055, ('RB', 'VBN'): 0.086272135613743, ('VBN', 'TO'): 0.10497585555322275, ('TO', 'VB'): 0.5837433576067703, ('VB', 'DT'): 0.2109024430779458, ('DT', 'JJ'): 0.19890919007362967, ('JJ', 'NN'): 0.4580053496369889, ('IN', 'NN'): 0.111535758214725, ('NN', 'NNS'): 0.08495040965933592, ('NN

In [81]:
######################
## LIST OF POS TAGS ##
######################ui
## The Viterbi trellis is stored as a 2D numpy array where one dimension is the number of POS tags, and the 
## other is the number of tokens in the input sentence. We want to be able to map columns in the array to POS tags. 
## Inspect the list of tags and the number of distinct tags. What tagset may it be? Why the two extra tags (BOS, EOS)?

pos2int = list(emits.keys())+["OOV"]
print(pos2int) 
print(len(pos2int))

['BOS', 'NN', 'IN', 'DT', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'JJ', 'NNS', 'NNP', ',', 'CC', 'POS', '.', 'EOS', 'VBP', 'VBG', 'PRP$', 'CD', '``', "''", 'VBD', 'EX', 'MD', '#', '(', '$', ')', 'NNPS', 'PRP', 'JJS', 'WP', 'RBR', 'JJR', 'WDT', 'WRB', 'RBS', 'PDT', 'RP', ':', 'FW', 'WP$', 'SYM', 'UH', 'OOV']
47


In [139]:
import nltk
########################
### VITERBI FUNCTION ###
########################

def run_viterbi(insent):

    ## Fill in some value for every state. But sometimes the emission probability will be 0 (e.g., 
    ## the probability that you see "the" when the tag is NN). Account for this possibility *and* for the 
    ## possibility of an OOV, by assigning defaultprob if the emission probability is 0.
    
    defaultprob =  0.00000000001

    ## Create the trellis and backpointer itself as a 2D np array
    stateprobs = np.zeros(shape=(len(insent), len(pos2int)))
    backpointers = np.zeros(shape=(len(insent), len(pos2int)),dtype=np.dtype(np.int32))

    ## Initialize state 0, which must be the beginning of sentence (BOS) tag with probability 1
    stateprobs[0,pos2int.index("BOS")] = 1.0

    ## Fill in the rest of the trellis, starting with state 1
    ## Populate both state probabilities and backpointers
    
    tempprevpos=[]
    pos=''
    word=''
    prevpos=['BOS']
    tempstate,bp=[],[]
    invocabflag=False
    emissionprob=0.0
    for w in range(1, len(insent)):
        word=insent[w]
#         print(word)
        invocabflag=False
        for p in range(0,len(pos2int)):
            pos=pos2int[p]
            tempstate=[]
            if(pos in emits and word in emits[pos]):
                invocabflag=True

#                 invocabflag=True
                prevword=insent[w-1]
                for prev in prevpos:   
                    # this is when 'OOV'=>pos
                    if((prev,pos) in transitions):
                            invocabflag=True
                            emissionprob=emits[pos][word]
                            tempprevpos.append(pos)
                            tempstate.append(transitions[prev,pos]*emissionprob*stateprobs[w-1,pos2int.index(prev)])
                
                statemax=max(tempstate)
                maxstate=tempstate.index(statemax)
#                 print(prevpos)
#                 print(maxstate)
#                 print((tempstate))
                partofspeech=prevpos[maxstate]
                backpointers[w,p]=pos2int.index(partofspeech)
                stateprobs[w,p]=statemax
                
        
        # when current word is OOV
        if(not invocabflag):
            for prev in prevpos:
                tempprevpos.append('OOV')
                tempstate.append(transitions[prev,'OOV']*defaultprob*stateprobs[w-1,pos2int.index(prev)])
            statemax=max(tempstate)
            maxstate=tempstate.index(statemax)
#             print(prevpos)
#             print(maxstate)
#             print(len(tempstate))

            partofspeech=prevpos[maxstate]
            backpointers[w,pos2int.index('OOV')]=pos2int.index(partofspeech)
            stateprobs[w,pos2int.index('OOV')]=statemax

#         print(stateprobs)
            
#         stateprobs=[]
        prevpos=tempprevpos[:]
        tempprevpos=[]

   
    ## Store the string of POS tags in a variable called posseq
    posseq = "EOS"

    ## After populating the full trellis and backpointer trellis, print out the proposed POS tag sequence by traversing
    ## the backpointers from the backpointer in the last column that corresponds to "EOS" (end of sentence tag).

    ## Getting started
    maxprob = stateprobs[len(insent)-1,pos2int.index("EOS")]
    maxprobid = backpointers[len(insent)-1,pos2int.index("EOS")]
#     print(backpointers)

    for i in range(len(insent)-2, 0, -1):
#         for
#         print(stateprobs)
#         print(i)
#         print(maxprobid)
        maxprobid=backpointers[i,maxprobid]
        posseq=pos2int[maxprobid]+" "+posseq+" "
#         maxprobid=backpointers[i,pos]
#         [5 0 0 0 0]<i,pos, pos=bp[i,pos]
#         [0 0 0 0 1]<i,5

    return posseq


In [140]:

######################
### TEST ON UNSEEN SENTENCES ###
######################
import nltk
from collections import defaultdict
## Function to find incorrect tags
def countcorrecttags(sq1, sq2):
    numcorrect = 0
    numwrong = 0
    if len(sq1) == len(sq2):
        for idx in range(len(sq1)):
            if sq1[idx] == sq2[idx]:
                numcorrect += 1
            else:
                numwrong += 1
    return [numcorrect, numwrong]
                
## Read in a list of 100 sentences and the correct POS tags. Use your function to generate POS tags for each sentence, then 
## compare the output against the correct POS tags.


## Use these to keep track of how many tags you got right.
V_numcorrect = 0
V_numwrong = 0

## Read in the test file of sentences to tag.
  #Q2
classfrequency=defaultdict(int)
        #/Q2
f = open("test-ps1.txt")
for line in f:
    
    ## Break each sentence into POS tags and word tokens
    line = re.sub(r'\(', "", line.rstrip())
    line = re.sub(r'\)', "", line.rstrip())
    parts = line.split()

    ## Create strings with words and POS tags
    testin = "BOS "
    testout = "BOS "
    for i in range(1, len(parts), 2):
        testout = testout + parts[i] + " "
        testin = testin + parts[i-1] + " "
    testout = testout + "EOS"
    testin = testin + "EOS"

    # print out the sentence, correct POS tags, proposed POS tags
    print(testin)
    print(testout)
#     ####Q2
#     for word in testin.split():
#         test=nltk.pos_tag([word])
#         classfrequency[test[0][1]]+=1
#     #####Q2
    ## Call the viterbi algorithm which returns a strong of POS tags
    viterbiguess = run_viterbi(testin.split())
    print(viterbiguess + "\n")

    ## Count correct and incorrect tags for this sentence
    sq1 = testout.split()
    sq2 = viterbiguess.split()
    (c, w) = countcorrecttags(sq1, sq2)
    V_numcorrect += c
    V_numwrong += w

f.close()

##Q2
count=0
for f in classfrequency:
    count+=classfrequency[f]

max_key = max(classfrequency, key=classfrequency.get)
print(classfrequency[max_key]/count)
###Q2
# Print out overall tagging accuracy
print("Viterbi Accuracy: ", (V_numcorrect / (V_numcorrect+V_numwrong)))

BOS Mr. Meador had been executive vice president of Balcor . EOS
BOS NNP NNP VBD VBN JJ NN NN IN NNP . EOS
BOS NNP OOV VBD VBN JJ NN NN IN OOV EOS          

BOS Shearson is about 60%-held by American Express Co . EOS
BOS NNP VBZ IN JJ IN NNP NNP NNP . EOS
BOS NNP VBZ IN OOV IN NNP NNP NNP EOS         

BOS Manville is a building and forest products concern . EOS
BOS NNP VBZ DT NN CC NN NNS VBP . EOS
BOS OOV VBZ DT NN CC NN NNS VBP EOS         

BOS Mr. Percival declined to comment . EOS
BOS NNP NNP VBD TO VB . EOS
BOS NNP OOV VBD TO VB EOS      

BOS The results were in line with analysts ' expectations . EOS
BOS DT NNS VBD IN NN IN NNS POS NNS . EOS
BOS DT NNS VBD IN NN IN NNS POS NNS EOS          

BOS The drug was introduced in West Germany this year . EOS
BOS DT NN VBD VBN IN NNP NNP DT NN . EOS
BOS DT NN VBD VBN IN NNP NNP DT NN EOS          

BOS Confectionery products sales also had strong growth in the quarter . EOS
BOS JJ NNS NNS RB VBD JJ NN IN DT NN . EOS
BOS OOV NNS NNS RB

ValueError: max() arg is an empty sequence